# Extracción de Coeficientes LPCC
Este notebook se centra en extraer los coeficientes LPC y convertirlos a LPCC (Cepstral Predictive Linear Coding) de una señal de audio.

## Índice:

1. Importación de Bibliotecas
2. Definición de Funciones para la Extracción de Coeficientes LPCC
3. Cargar Datos de Audio
4. Extracción de Coeficientes LPC
5. Conversión de LPC a LPCC
6. Guardado de Coeficientes
7. Resumen de los Resultados

## 1. Importación de Bibliotecas
Se importan las bibliotecas necesarias para la extracción y procesamiento de los coeficientes LPC y LPCC.

In [ ]:
# numpy es una biblioteca de Python utilizada para realizar operaciones matemáticas
# sobre arrays y matrices de manera eficiente. Se abrevia comúnmente como np.
import numpy as np

# librosa es una biblioteca de Python para el análisis de audio y música. 
# Proporciona las herramientas necesarias para extraer información de señales de audio
# tales como tiempo de duración, frecuencia, entre otras.
import librosa

## 2. Definición de Funciones para la Extracción de Coeficientes LPCC
Se presentan las funciones que se usarán para cargar los datos de audio y extraer/guardar los coeficientes LPCC.

### 2.1. Función para cargar frames de audio desde un archivo
Carga frames de audio de un archivo utilizando la función `load` de numpy.

>**Parámetros:**
* **archivo_entrada (str):** Ruta o nombre del archivo que contiene los frames de audio a cargar.

>**Retorna:**
* **ndarray:** Un array de numpy con los frames de audio cargados del archivo.

In [ ]:
def cargar_frames_desde_archivo(archivo_entrada):
    return np.load(archivo_entrada)

### 2.2. Función para extraer coeficientes LPC
Extrae los coeficientes LPC (Linear Predictive Coding) de cada frame de audio. Se extraen los coeficientes utilizando la función [lpc()](https://librosa.org/doc/main/generated/librosa.lpc.html) de la biblioteca [librosa](https://librosa.org/doc/latest/index.html).

>**Parámetros:**
* **frames (ndarray):** Una matriz donde cada columna representa un frame de audio.
* **n_coeficientes (int):** Número de coeficientes LPC a extraer para cada frame.

>**Retorna:**
* **ndarray:** Una matriz donde cada fila contiene los coeficientes LPC (excluyendo el primer coeficiente) de un frame.

>**Nota:**
El primer coeficiente de LPC, que usualmente es 1.0 para señales normalizadas, se excluye porque no ofrece información relevante para la mayoría de aplicaciones de análisis.

In [ ]:
def extraer_coeficientes_lpc(frames, n_coeficientes):
    coeficientes_lpc = []

    for frame in frames.T:
        coef_lpc_frame = librosa.lpc(frame, order=n_coeficientes)
        coef_lpc_frame = coef_lpc_frame[1:]  # Excluir el primer coeficiente (1.0)
        coeficientes_lpc.append(coef_lpc_frame)
    return np.array(coeficientes_lpc)

### 2.3. Función para convertir coeficientes LPC a LPCC
Convierte los coeficientes LPC (Linear Predictive Coding) a LPCC (Linear Predictive Cepstral Coefficients).

>**Parámetros:**
* **coeficientes_lpc (ndarray):** Una matriz donde cada fila contiene los coeficientes LPC de un frame.
* **n_lpcc (int):** Número de coeficientes LPCC a calcular para cada frame.

>**Retorna:**
* **ndarray:** Una matriz donde cada fila contiene los coeficientes LPCC de un frame.

>**Nota:**
Los coeficientes LPCC son una representación cepstral de los coeficientes LPC y suelen ser utilizados en aplicaciones de procesamiento de voz.

**IMPORTANTE: ** Para realizar la conversión de coeficientes LPC a LPCC se tomó como referencia la siguiente ecuación: 

La relación entre los coeficientes LPCC (C<sub>m</sub>) y LPC (a<sub>m</sub>) puede ser expresada por la ecuación:

$$C_m = a_m + \sum_{k=1}^{m-1} \frac{k}{m} C_k a_{m-k} \quad$$

para $1 < m < p$, y $k=1,2,3,\ldots,p$ donde $C_m$ son los coeficientes LPCC, $a_m $ son los coeficientes LPC y $p$ es el orden de los coeficientes. En nuestro caso, $p = 13$.

Se utilizó la siguiente referencia:

> K. S. Rao, V. Ramu Reddy, and S. Maity, Language identification using spectral and prosodic features. 
Springer International Publishing, 2015

In [ ]:
def lpc_a_lpcc(coeficientes_lpc, n_lpcc):
    coeficientes_lpcc = []

    for frame_lpc in coeficientes_lpc:
        frame_lpcc = np.zeros(n_lpcc)

        frame_lpcc[0] = frame_lpc[0]

        for m in range(1, n_lpcc):
            sumatoria = 0
            for k in range(1, m):
                sumatoria += (k / m) * frame_lpcc[k] * frame_lpc[m - k]
            frame_lpcc[m] = frame_lpc[m] + sumatoria

        coeficientes_lpcc.append(frame_lpcc)

    return np.array(coeficientes_lpcc)

## 3. Cargar Datos de Audio
Aquí se carga la señal de audio desde un archivo .npy.

In [ ]:
nombre_archivo_frames_lpc = "nombre_del_archivo_que_contiene_los_frames.npy"
frames = cargar_frames_desde_archivo(nombre_archivo_frames_lpc)

## 4. Extracción de Coeficientes LPC
A partir de la señal de audio cargada, se extraen los coeficientes LPC.

In [ ]:
n_coeficientes_lpc = 14
coeficientes_lpc = extraer_coeficientes_lpc(frames, n_coeficientes_lpc)

## 5. Conversión de LPC a LPCC
A partir de los coeficiente LPC se obtienen los coeficientes LPCC.

In [ ]:
n_coeficientes_lpcc = 13
coeficientes_lpcc = lpc_a_lpcc(coeficientes_lpc, n_coeficientes_lpcc)

## 5. Guardado de Coeficientes
Una vez extraídos los coeficientes LPCC, se procede a guardarlos en un archivo .npy.

In [ ]:
# Guardar coeficientes LPCC
archivo_coeficientes_lpcc = 'coeficientes_lpcc.npy'
np.save(archivo_coeficientes_lpcc, coeficientes_lpcc)

## 7. Resumen de los Resultados
1. Archivo original de frames: **nombre_del_archivo_que_contiene_los_frames.npy**
2. Número de coeficientes LPCC por frame: 13
3. Archivo de salida: **coeficientes_lpcc.npy (para LPCC)**